In [5]:
import requests
import pandas as pd

# API 요청을 자동화하여 데이터 가져오기
def get_data(start, end):
    url = f'http://openapi.seoul.go.kr:8088/5648446357726b6437355358536443/json/GarakGradePrice/{start}/{end}/'
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data['GarakGradePrice']['row'])

df = pd.DataFrame()

# 1000개씩 데이터 가져와서 합치기
for i in range(1, 3000, 1000):
    data = get_data(i, i + 999)
    df = pd.concat([df, data])

# 인덱스 재설정
df.reset_index(drop=True, inplace=True)

# 결과 출력
df = df[~((df['MAXPRICE']==0) & (df['MAXPRICE']==0) & (df['MAXPRICE']==0))]# 최고가 0인것들 제외(이상치)
df['kg당 평균 가격'] = (df['AVGPRICE'] / df['UNITQTY']).astype('int')
df

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격
0,(냉)갈치,상,10.0,kg상자,350000.0,26667.0,90770.0,20240313,9077
1,(냉)갈치,중,10.0,kg상자,26667.0,25000.0,26276.0,20240313,2627
2,(냉)고등어,상,10.0,kg상자,80000.0,27000.0,40154.0,20240313,4015
4,(냉)고등어 수입,상,10.0,kg상자,55000.0,34000.0,35581.0,20240313,3558
8,(선)갈치,상,5.0,kg상자,190000.0,180000.0,184375.0,20240313,36875
...,...,...,...,...,...,...,...,...,...
2011,활 방어(자연),중,1.0,kg,8800.0,4700.0,5569.0,20240313,5569
2012,활 방어(자연),소,1.0,kg,4700.0,1300.0,3684.0,20240313,3684
2017,활 전복(양식),상,1.0,kg,32000.0,20000.0,24602.0,20240313,24602
2018,활 전복(양식),중,1.0,kg,20000.0,8500.0,16359.0,20240313,16359


In [7]:
def category(item):
  su_list = ["갈치", "고등어", "멸치", "조개", "게", "새우", "오징어", "낙지", "문어", "해삼", "고래", "바다", "물고기" , '활' , '가자미' , '명태' , '어','홍합','소라','굴',
             "근대", "깐 바지락", "냉태", "대구", "달래", "매생이", "머위대", "삼치",'조기' ,"부세", "주꾸미",'다시마']

  ve_list = ["가지", "감자", "양파", "배추", "무", "당근", "토마토", "오이", "버섯", "고추", "피망", "깻잎", "시금치", "열무", "콜리플라워", "브로콜리", "아스파라거스",
              "양배추", "샐러리", "케일", "깻잎", "깻잎순", "쑥갓" ,'고구마','고수','로메인','호박','미나리','콜라비','미더덕','아귀','칼리',"김", 
              "깐마늘", "냉이", "노랑 파프리카", "당귀잎", "대파", "동죽", "레드 치커리","생표고", "나물" , '파'
                 "로케트 루꼴라", "매생이", "머위대", "바실",  "부추", "블루베리", "비타민","비트", "빨강 파프리카", "상추", "생표고", "셀러리", "아욱","양송이", "부추",
                 "오만둥이", "적근대", "적상추", "쪽파", "쫑상추",  "참두릅", "청경채",
                 "취나물", "치커리",'루꼴라']
  fru_list = ['귤','배','사과','샤인머스캣','파인애플','수박','딸기','멜론','바나나',"단감", "만감", "참외", "오렌지"]

  # 품목명에 키워드 포함 여부 확인
  for keyword in su_list:
    if keyword in item:
      return "수산물"
  for keyword in ve_list:
    if keyword in item:
      return "야채"
  for keyword in fru_list:
    if keyword in item:
      return "과일"
  return "기타"

df['category'] = df['PUMNAME'].apply(category)
df

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격,category
0,(냉)갈치,상,10.0,kg상자,350000.0,26667.0,90770.0,20240313,9077,수산물
1,(냉)갈치,중,10.0,kg상자,26667.0,25000.0,26276.0,20240313,2627,수산물
2,(냉)고등어,상,10.0,kg상자,80000.0,27000.0,40154.0,20240313,4015,수산물
4,(냉)고등어 수입,상,10.0,kg상자,55000.0,34000.0,35581.0,20240313,3558,수산물
8,(선)갈치,상,5.0,kg상자,190000.0,180000.0,184375.0,20240313,36875,수산물
...,...,...,...,...,...,...,...,...,...,...
2011,활 방어(자연),중,1.0,kg,8800.0,4700.0,5569.0,20240313,5569,수산물
2012,활 방어(자연),소,1.0,kg,4700.0,1300.0,3684.0,20240313,3684,수산물
2017,활 전복(양식),상,1.0,kg,32000.0,20000.0,24602.0,20240313,24602,수산물
2018,활 전복(양식),중,1.0,kg,20000.0,8500.0,16359.0,20240313,16359,수산물


In [8]:
def get_category_list(df):
    etc_lst = []
    for i in range(len(df)):
        if df['category'].iloc[i] == '기타':
            etc_lst.append(df['PUMNAME'].iloc[i])

    return etc_lst

In [9]:
get_category_list(df)

['그린키위 국산',
 '그린키위 국산',
 '그린키위 국산',
 '그린키위 국산',
 '머위잎',
 '머위잎',
 '머위잎',
 '파슬리',
 '파슬리',
 '파슬리']

In [10]:
df[df['category']=='기타']

,PUMNAME,GRADENAME,UNITQTY,UNITNAME,MAXPRICE,MINPRICE,AVGPRICE,INVEST_DT,kg당 평균 가격,category
278,그린키위 국산,특,10.0,kg상자,54000.0,54000.0,54000.0,20240313,5400,기타
279,그린키위 국산,상,10.0,kg상자,54000.0,53000.0,53480.0,20240313,5348,기타
280,그린키위 국산,중,10.0,kg상자,53000.0,48000.0,51378.0,20240313,5137,기타
281,그린키위 국산,하,10.0,kg상자,48000.0,22000.0,36125.0,20240313,3612,기타
781,머위잎,상,4.0,kg상자,67500.0,36000.0,41555.0,20240313,10388,기타
782,머위잎,중,4.0,kg상자,36000.0,30000.0,32535.0,20240313,8133,기타
783,머위잎,하,4.0,kg상자,30000.0,24000.0,28654.0,20240313,7163,기타
1922,파슬리,상,4.0,kg상자,14000.0,11000.0,13385.0,20240313,3346,기타
1923,파슬리,중,4.0,kg상자,11000.0,7000.0,8783.0,20240313,2195,기타
1924,파슬리,하,4.0,kg상자,7000.0,6500.0,6717.0,20240313,1679,기타


## EDA

In [12]:
df = df[~(df['최고가']==0)] # 최고가 0인것들 제외(이상치)

In [41]:
# kg당 평균 가격
df['kg당 평균 가격'] = (df['평균가'] / df['수량']).astype('int')
df['kg당 평균 가격'] 

C:\Users\administ\ipykernel_17052\2162443189.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kg당 평균 가격'] = (df['평균가'] / df['수량']).astype('int')


0       12519
1        2475
2        2766
4        3134
8       25940
        ...  
2042    16560
2043    13695
2044    16074
2045    15152
2046    12041
Name: kg당 평균 가격, Length: 818, dtype: int32

In [42]:
# 품목별 평균가 최소 메물
df.groupby('품목명')['kg당 평균 가격'].min()

품목명
(냉)갈치         2475
(냉)고등어        2766
(냉)고등어 수입     3134
(선)갈치         4000
(선)고등어        3081
             ...  
활 민어(자연)     12080
활 방어(자연)      6354
활 우럭(자연)      7927
활 전복(양식)     17906
활 점성어 수입     12471
Name: kg당 평균 가격, Length: 209, dtype: int32

In [43]:
# 품목별 kg당 평균 가격 평균(그날의 평균 kg당 시세)
df.groupby('품목명')['kg당 평균 가격'].mean().sort_values()

품목명
시금치 포항초          5.500000
부추(일반)           7.000000
영양부추            16.666667
깻잎             257.000000
무              545.250000
                ...      
딸기 죽향        31228.250000
참두릅(자연산)     36682.333333
건오징어 근해      55140.000000
블루베리 국산      95221.500000
바실          104788.666667
Name: kg당 평균 가격, Length: 209, dtype: float64

In [21]:
df['품목명'].value_counts() 

품목명
양파          16
만감 천혜향      16
만감 한라봉      12
생표고         12
양상추(일반)      9
            ..
갑오징어         1
활 점성어 수입     1
대파 수입        1
대구 냉장 수입     1
냉동 낙지 수입     1
Name: count, Length: 209, dtype: int64

In [22]:
df['등급'].value_counts() 

등급
상    241
중    234
하    205
특    120
대     10
소      8
Name: count, dtype: int64

In [23]:
df['품목명'].unique()

array(['(냉)갈치', '(냉)고등어', '(냉)고등어 수입', '(선)갈치', '(선)고등어', '(선)명태 수입',
       '가무락(모시조개)', '가자미', '가지', '감귤', '감귤 금귤', '감귤 비가림', '감귤 하우스',
       '사과 후브락스', '삼치', '상추', '상추 적포기', '상추 포기찹', '감숭어', '감자', '감자 대지마',
       '감자 두백', '감자 수미', '감자 수미(햇)', '갑오징어', '건대멸치', '건세멸치', '건소멸치',
       '건오징어 근해', '건자멸치', '건중멸치', '겉홍합', '고구마', '고구마 호풍미', '고수', '녹광고추',
       '느타리버섯', '늙은호박', '단감 부유', '단호박 수입', '달래(일반)', '당귀잎', '당근',
       '골드파인애플 수입', '곰취나물', '굴', '그린키위 국산', '근대', '김 개량', '깐 바지락',
       '깐마늘 대서', '깐쪽파', '깻잎', '깻잎순', '꽃게 수입', '꽃느타리버섯', '꽈리고추', '낙지 수입',
       '냉동 가자미 수입', '냉동 낙지 수입', '냉동 오징어(연안)', '냉이', '냉태 원양', '노랑 파프리카',
       '배추', '배추얼갈이', '백다다기오이', '백조기', '병어', '당근 수입', '대구', '대구 냉장 수입',
       '대추방울토마토', '대파 수입', '대파(일반)', '돌미나리', '돗나물', '딸기', '딸기 금실',
       '딸기 메리퀸', '딸기 설향', '딸기 장희', '딸기 죽향', '딸기 킹스베리', '땅두릅(재배산)',
       '레드 치커리', '로메인(일반)', '로케트 루꼴라', '롱그린고추', '봄동배추', '봉지 미역', '부추(일반)',
       '만가닥버섯', '만감 천혜향', '만감 한라봉', '맛느타리버섯', '매생이(일반)', '머위대', '멍게',
       '멜론 머스크', '

In [ ]:
def category(item):
  su_list = ["갈치", "고등어", "멸치", "조개", "게", "새우", "오징어", "낙지", "문어", "해삼", "고래", "바다", "물고기" , '활' , '가자미' , '명태' , '어','홍합','소라','굴',
             "근대", "깐 바지락", "냉태", "대구", "달래", "매생이", "머위대", "삼치",'조기' ,"부세", "주꾸미"]

  ve_list = ["가지", "감자", "양파", "배추", "무", "당근", "토마토", "오이", "버섯", "고추", "피망", "깻잎", "시금치", "열무", "콜리플라워", "브로콜리", "아스파라거스",
              "양배추", "샐러리", "케일", "깻잎", "깻잎순", "쑥갓" ,'고구마','고수','로메인','호박','미나리','콜라비','미더덕','아귀','칼리',"김", 
              "깐마늘", "냉이", "노랑 파프리카", "당귀잎", "대파", "동죽", "레드 치커리","생표고", "나물" , '파'
                 "로케트 루꼴라", "매생이", "머위대", "바실",  "부추", "블루베리", "비타민",
                 "비트", "빨강 파프리카", "상추", "생표고", "셀러리", "아욱","양송이", "부추",
                 "오만둥이", "적근대", "적상추", "쪽파", "쫑상추",  "참두릅", "청경채",
                 "취나물", "치커리",'루꼴라']
  fru_list = ['귤','배','사과','샤인머스캣','파인애플','수박','딸기','멜론','바나나',"단감", "만감", "참외", "오렌지"]

  # 품목명에 키워드 포함 여부 확인
  for keyword in su_list:
    if keyword in item:
      return "수산물"
  for keyword in ve_list:
    if keyword in item:
      return "야채"
  for keyword in fru_list:
    if keyword in item:
      return "과일"
  return "기타"

In [2]:
df['품목명'].value_counts()

품목명
수박(일반)         28
양파             20
복숭아 유명백도(백)    16
키위 기타 수입       16
만감 한라봉         16
               ..
활 우럭(자연)        1
바지락             1
활 미꾸라지(양식)      1
땅콩 수입           1
대구 냉장 수입        1
Name: count, Length: 425, dtype: int64

In [3]:
df['등급'].value_counts()

등급
상    601
중    562
하    494
특    359
대     18
소     13
Name: count, dtype: int64